In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Load tidied dataframes

# spells_df = pd.read_pickle('data/spells_df')
monster_df = pd.read_pickle('data/monster_df')
actions_df = pd.read_pickle('data/actions_df')
spellbook_df = pd.read_pickle('data/spellbook_df')
innate_spellbook_df = pd.read_pickle('data/innate_spellbook_df')
# special_abilities_df = pd.read_pickle('data/special_abilities_df')
# condition_immunities_df = pd.read_pickle('data/condition_immunities_df')
# damage_immunities_df = pd.read_pickle('data/damage_immunities_df')
# damage_resistances_df = pd.read_pickle('data/damage_resistances_df')
# damage_vulnerabilities_df = pd.read_pickle('data/damage_vulnerabilities_df')

# `actions_df`

In [3]:
actions_df = actions_df.reset_index('action')

In [82]:
# ill-formed multiattack statuses

outliers = ['Animated Armor',
            'Cult Fanatic',
            'Deva',
            'Doppelganger',
            'Erinyes',
            'Gibbering Mouther',
            'Gladiator',
            'Iron Golem',
            'Knight',
            'Lizardfolk',
            'Planetar',
            'Scout',
            'Spy',
            'Thug',
            'Violet Fungus',
            'Werebear']

In [73]:
a = (actions_df.index).difference(outliers)
df = actions_df.loc[a]

In [81]:
# actions_df[actions_df.index.isin(outliers)]

# multiattack

In [50]:
def normalize_multiattack(x):
    df = x.set_index('action').copy()
    if 'Multiattack' in df.index:
        desc = df.loc['Multiattack'].desc.lower().strip()
        actions = [format_action(action) for action in df.index
                   if action != 'Multiattack']
        multiattack, exception = extract_attacks(desc, actions)
        return pd.DataFrame({'desc': [desc],
                             'multiattack': [multiattack],
                             'exception': [exception]})
    return


def format_action(action):
    action = action.lower()
    i = action.find('(')
    if i > 0:
        return action[:i-1]
    else:
        return action


def extract_attacks(desc, actions):
    exceptions_pattern = r'\b({})\b'.format('|'.join(x for x in exceptions))
    if re.search(exceptions_pattern, desc):
        exception = True
    else:
        exception = False
    split_pattern = r'.*?({})'.format('|'.join(x for x in actions))
    actions_pattern = '|'.join(x for x in actions)
    numerics_pattern = '|'.join(x for x in numerics.keys())
    multiattack = list()
    for x in re.finditer(split_pattern, desc):
        start, end = x.span()
        x = re.findall(numerics_pattern, desc[start:end])
        if x:
            n = numerics[x[-1]]
        else:
            n = 1
        y = re.findall(actions_pattern, desc[start:end])
        multiattack.extend(n * y)
    return multiattack, exception

numerics = {'one': 1,
            'two': 2,
            'twice': 2,
            'three': 3,
            'four': 4,
            'six': 6,
            'seven': 7,
            }
exceptions = ('or', 'alternatively', 'when', 'in place', 'replace', 'also', 'as many', 'only')

In [67]:
multiattacks_df = df.groupby('name').apply(normalize_multiattack)
multiattacks_df.reset_index(level=1, drop=True, inplace=True)

We'll correct the following by hand

In [68]:
multiattacks_df[multiattacks_df.exception]

,desc,exception,multiattack
name,,,
Bandit Captain,the captain makes three melee attacks: two wit...,True,"[scimitar, scimitar, dagger, dagger, dagger]"
Barbed Devil,the devil makes three melee attacks: one with ...,True,"[tail, claw, claw, hurl flame]"
Centaur,the centaur makes two attacks: one with its pi...,True,"[pike, hooves, longbow, longbow]"
Chimera,the chimera makes three attacks: one with its ...,True,"[bite, horns, claws, fire breath, bite, horns]"
Chuul,the chuul makes two pincer attacks. if the chu...,True,"[pincer, pincer, tentacles]"
Dragon Turtle,the dragon turtle makes three attacks: one wit...,True,"[bite, claw, claw, tail, claw, claw]"
Drider,"the drider makes three attacks, either with it...",True,"[longsword, longsword, longsword, longbow, bite]"
Efreeti,the efreeti makes two scimitar attacks or uses...,True,"[scimitar, scimitar, hurl flame]"
Glabrezu,the glabrezu makes four attacks: two with its ...,True,"[pincer, pincer, fist, fist, pincer, pincer]"


There are lots of special exceptions to deal with?

# other actions

In [10]:
for x in actions_df[['action', 'desc']].iterrows():
    print(x[0])
    print(x[1].desc)

Aboleth
The aboleth makes three tentacle attacks.
Aboleth
Melee Weapon Attack: +9 to hit, reach 10 ft., one target. Hit: 12 (2d6 + 5) bludgeoning damage. If the target is a creature, it must succeed on a DC 14 Constitution saving throw or become diseased. The disease has no effect for 1 minute and can be removed by any magic that cures disease. After 1 minute, the diseased creature's skin becomes translucent and slimy, the creature can't regain hit points unless it is underwater, and the disease can be removed only by heal or another disease-curing spell of 6th level or higher. When the creature is outside a body of water, it takes 6 (1d12) acid damage every 10 minutes unless moisture is applied to the skin before 10 minutes have passed.
Aboleth
Melee Weapon Attack: +9 to hit, reach 10 ft. one target. Hit: 15 (3d6 + 5) bludgeoning damage.
Aboleth
The aboleth targets one creature it can see within 30 ft. of it. The target must succeed on a DC 14 Wisdom saving throw or be magically charm

Melee Weapon Attack: +6 to hit, reach 5 ft., one creature. Hit: 2 (1d4) piercing damage plus 9 (2d8) poison damage.
Drider
Melee Weapon Attack: +6 to hit, reach 5 ft., one target. Hit: 7 (1d8 + 3) slashing damage, or 8 (1d10 + 3) slashing damage if used with two hands.
Drider
Ranged Weapon Attack: +6 to hit, range 150/600 ft., one target. Hit: 7 (1d8 + 3) piercing damage plus 4 (1d8) poison damage.
Drow
Melee Weapon Attack: +4 to hit, reach 5 ft., one target. Hit: 5 (1d6 + 2) piercing damage.
Drow
Ranged Weapon Attack: +4 to hit, range 30/120 ft., one target. Hit: 5 (1d6 + 2) piercing damage, and the target must succeed on a DC 13 Constitution saving throw or be poisoned for 1 hour. If the saving throw fails by 5 or more, the target is also unconscious while poisoned in this way. The target wakes up if it takes damage or if another creature takes an action to shake it awake.
Druid
Melee Weapon Attack: +2 to hit (+4 to hit with shillelagh), reach 5 ft., one target. Hit: 3 (1d6) bludgeon

The manticore makes three attacks: one with its bite and two with its claws or three with its tail spikes.
Manticore
Melee Weapon Attack: +5 to hit, reach 5 ft., one target. Hit: 7 (1d8 + 3) piercing damage.
Manticore
Melee Weapon Attack: +5 to hit, reach 5 ft., one target. Hit: 6 (1d6 + 3) slashing damage.
Manticore
Ranged Weapon Attack: +5 to hit, range 100/200 ft., one target. Hit: 7 (1d8 + 3) piercing damage.
Marilith
The marilith can make seven attacks: six with its longswords and one with its tail.
Marilith
Melee Weapon Attack: +9 to hit, reach 5 ft., one target. Hit: 13 (2d8 + 4) slashing damage.
Marilith
Melee Weapon Attack: +9 to hit, reach 10 ft., one creature. Hit: 15 (2d10 + 4) bludgeoning damage. If the target is Medium or smaller, it is grappled (escape DC 19). Until this grapple ends, the target is restrained, the marilith can automatically hit the target with its tail, and the marilith can't make tail attacks against other targets.
Marilith
The marilith magically telepo

In [11]:
def damage_summary(number, size, bonus):
    mu = (size + 1) / 2
    mean = number * mu + bonus
    std = np.sqrt(number * (sum_of_squares(size)/size - mu ** 2))
    return mean, std

def sum_of_squares(n):
    return n*(2*n+1)*(n+1)/6

In [12]:
mean = 3.5
n = 6
np.sqrt(sum(x ** 2 / n for x in range(1,n+1)) - mean ** 2)

1.7078251276599334

In [13]:
damage_summary(1,6,0)

(3.5, 1.707825127659933)